# Firmware Diffing

## Introduction: Netgear RAX30

For this tutorial we are going to work on the Netgear RAX30 Wireless router, that was part of the targets for [Pwn2Own Toronto 2022](https://www.zerodayinitiative.com/blog/2022/8/29/announcing-pwn2own-toronto-2022-and-introducing-the-soho-smashup). A day before the submission deadline, Netgear
released a firmware update that was also patching some vulnerabilities (which led multiple Pwn2Own competitors to withdraw theirs submission). The two versions are:


The goal of this tutorial is to show how performing a full-filesystem diff and to navigate the results programatically.

## I. Unpacking firmwares

To extract the firmwares one can use [Unblob](https://unblob.org) which is now undeniably
the best firmware extraction tool to date. We can extract the firmware with:

```bash
docker run \
  --rm \
  --pull always \
  -v $PWD/extract/RAX30-V1.0.7.78_1.img_extract:/data/output \
  -v $PWD:/data/input \
  ghcr.io/onekey-sec/unblob:latest /data/input/RAX30-V1.0.7.78_1.img
```

*(Considering the firmware is in your current directory)*

**Exercise**: Unpack both firmwares

## II. Exporting files

Before diffing executables between the two firmwares, we have to call BinExport on all of them.
For that we can use the program ``binexporter`` that can iterate a whole tree of files and to
export all files encountered.

```bash
    binexporter -t 10 extract/RAX30-V1.0.7.78_1.img_extract
```

The option ``-t`` launch the export with 10 processes. As exporting implies disassembling all binaries it can take a while to perform (~1h for this firmware).

To continue the tutorial here are the binexports files for both firmwares.

## III. Performing the diff

Now that all executable files are exported in a single directory for each firmwares
we can perfom the diff of each BinExport files based on theirs name. *(For simplicity
we assume there are now duplicates. Also symlinks also led to duplicating binexport files)*.

**Exercise**: Write a script to diff all binexports files.

In [ ]:
from pathlib import Path
from bindiff import BinDiff, BindiffFile

binexports1 = Path("binexports/RAX30-V1.0.7.78_1.img_extract")
binexports2 = Path("binexports/RAX30-V1.0.9.90_3.img_extract")

diff_dir = Path("diffs")  # create output dir
diff_dir.mkdir()

for file1 in binexports1.iterdir():
    file2 = binexports2 / file1.name
    if file2.exists():
        diff_file = diff_dir / f"{file1.with_suffix('').name}_vs_{file2.with_suffix('').name}.BinDiff"
        if not diff_file.exists():
            print(f"diff {file1} | {file2}: ", end="")
            print(BinDiff.raw_diffing(file1, file2, diff_file))  # Perform the diff

## IV. Analyzing diffs

Now that all diff files are created we can open them to analyze changes.

**Exercise**: Write a script that take the 15 most different binaries (similarity the lowest), and which shows the 10 most different functions (dissimilar ones).

In [8]:
from ipywidgets import IntProgress
from IPython.display import display
from collections import Counter
from bindiff import BinDiff, BindiffFile

diff_dir = Path("diffs/")

progress = IntProgress(min=0, max=len(list(diff_dir.iterdir())), description="Load diffs")
display(progress)

diffs = Counter()
for diff_file in diff_dir.iterdir():
    progress.value += 1
    diff = BindiffFile(diff_file)
    diffs[diff_file] = diff.similarity
    del diff  # useful to free the memory

# Reopen the most dissimilar binaries
for diff_file, similarity in diffs.most_common()[::-1][:15]:
    diff = BindiffFile(diff_file)
    binary_file = str(diff_file.name).split("_vs_")[0]
    print(f"{binary_file}: {similarity:.0%} [matched:{len(diff.function_matches)}][unmatched:{diff.unmatched_primary_count}-{diff.unmatched_secondary_count}]:")
    fun_sims = Counter({x.name1: x.similarity for x in diff.function_matches})
    for fun, sim in fun_sims.most_common()[::-1][:10]:
        print(f"  - {fun} ({sim:.0%})")

IntProgress(value=0, description='Load diffs', max=965)

xCloud_Debug_log.zip: 49% [matched:140][unmatched:28-28]:
  - sub_000115D8 (10%)
  - sub_00013DBC (59%)
  - _ITM_registerTMCloneTable (100%)
  - __imp___gmon_start__ (100%)
  - _ITM_deregisterTMCloneTable (100%)
  - __imp_fwrite (100%)
  - __imp_json_object_object_add (100%)
  - __imp_strstr (100%)
  - __imp_snprintf (100%)
  - __imp_cmsLog_cleanup (100%)
debug.cgi: 49% [matched:140][unmatched:28-28]:
  - sub_000115D8 (10%)
  - sub_00013DBC (59%)
  - _ITM_registerTMCloneTable (100%)
  - __imp___gmon_start__ (100%)
  - _ITM_deregisterTMCloneTable (100%)
  - __imp_fwrite (100%)
  - __imp_json_object_object_add (100%)
  - __imp_strstr (100%)
  - __imp_snprintf (100%)
  - __imp_cmsLog_cleanup (100%)
libmdm_db.so: 51% [matched:258][unmatched:418-418]:
  - oalMdm_isParam64 (63%)
  - strstr (66%)
  - mdm_deleteObjectInstance (92%)
  - j_mdm_getObject (99%)
  - mdm_getNextObjPathDesc (99%)
  - sub_00005F68 (99%)
  - sub_0000B864 (99%)
  - mdm_moveInstanceUsingNewOrderValue (99%)
  - j_mdm_getO

## Bonus

The latest version released is [1.0.11.96](https://www.netgear.fr/support/product/rax30#download) which apparently provide a **"hot fix"**. Can you identify what has been patched and how ?